## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP2

## Objectifs d'apprentissage: 

•	Explorer les modèles d’espace vectoriel (vector space models) comme représentations distribuées de la sémantique des mots 
•	Implémenter la fréquence de co-occurrence et la PPMI
•	Comprendre différentes mesures de distance entre vecteurs de mots 
•	Explorer l’intérêt de la réduction de dimensionnalité 



## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Equipe 8:

Cedric Sadeu (1869737): 1/3

Mamoudou Sacko (1924187): 1/3

Oumayma Messoussi (2016797): 1/3



## Support de google Colab



In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!wget https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz

--2020-10-07 01:13:15--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  12.6MB/s    in 7.7s    

2020-10-07 01:13:22 (10.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

--2020-10-07 01:13:23--  https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz
Resolving staff.fnwi.uva.nl (staff.fnwi.uva.nl)... 146.50.61.62
Connecting to staff.fnwi.uva.nl (staff.fnwi.uva.nl)|146.50.61.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90373 (88K) [application/x-gzip]
Saving to: ‘MEN.tar.gz’

MEN.tar.gz          100%[===================>]  88.25K   239KB/s    in 0.4s    

2020-10-07 01:13:24 (239 KB/s) - ‘MEN.tar.gz’ saved [90373/90373]


In [ ]:
! tar -xzf aclImdb_v1.tar.gz
! tar -xzf MEN.tar.gz
! mkdir -p vsm

## Librairies externes

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from collections import Counter, defaultdict
from itertools import chain
from bs4 import BeautifulSoup
import csv
import itertools
import numpy as np
import os
import pandas as pd
import glob
import re
import math
import time

from nltk.corpus import stopwords as all_stopwords
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter, defaultdict

from scipy.stats import spearmanr
from scipy.spatial.distance import euclidean, cosine
from IPython.display import display
from sklearn.decomposition import TruncatedSVD

## Valeurs globales

In [ ]:
DIRNAME_ACL =  os.path.join(os.getcwd(), "aclImdb/")
DIRNAME_MEN =  os.path.join(os.getcwd(), "MEN/")
DIRNAME_VSM =  os.path.join(os.getcwd(), "vsm")

## 1. Prétraitement (20 points)

**a)**	Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données. Commencez par lire ces données, en gardant séparées les données d'entraînement et de test. La fonction doit mettre les mots en minuscules,  supprimer les stopwords (vous devez utiliser ceux de NLTK) et afficher le nombre total de phrases d’entrainement,  le nombre total de phrases d’entrainement positives et négatives et le nombre total de phrases de test avec le nombre total de phrases de test positives et négatives ;

In [ ]:
train_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}
test_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}

for name in ['train/pos', 'train/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        train_data['id'].append(review_id)
        train_data['text'].append(content)
        train_data['rating'].append(rating)
        train_data['sentiment'].append(sentiment)
                
for name in ['test/pos', 'test/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        test_data['id'].append(review_id)
        test_data['text'].append(content)
        test_data['rating'].append(rating)
        test_data['sentiment'].append(sentiment)

train_df = pd.DataFrame (train_data, columns = ['id', 'text', 'rating', 'sentiment'])
test_df = pd.DataFrame (test_data, columns = ['id', 'text', 'rating', 'sentiment'])

In [ ]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,4630,This is the second Eytan Fox film I have seen....,0.9,pos
1,10964,"When it first came out, this work by the Meyse...",0.7,pos
2,4696,"even though this movie is quite old, no matter...",1.0,pos
3,4807,I really thought that this movie was superb. N...,1.0,pos
4,5258,"The Ladies Man is laugh out loud funny, with a...",1.0,pos


In [ ]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,5739,Paul Naschy as a ghostly security guard in thi...,0.7,pos
1,10964,"In many ways, the filmic career of independent...",0.7,pos
2,4807,Having seen 'only' about 200 Hong Kong films i...,1.0,pos
3,7259,This movie almost has everything. The action i...,1.0,pos
4,11405,"All you need is great house, a babysitter and ...",0.8,pos


In [ ]:
def preprocess(review):
    soup = BeautifulSoup(review, "html.parser")
    text = soup.get_text()
    stop_words = set(all_stopwords.words('english'))
    
    tokens = nltk.word_tokenize(text.lower())
    return [w for w in tokens if w.isalpha() and not w in stop_words] 

In [ ]:
start = time.time()
train_df['text'] = train_df['text'].apply(preprocess)
print(time.time() - start)

test_df['text'] = test_df['text'].apply(preprocess)

54.52512502670288


In [ ]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,4630,"[second, eytan, fox, film, seen, fantastic, ac...",0.9,pos
1,10964,"[first, came, work, meysels, brothers, much, c...",0.7,pos
2,4696,"[even, though, movie, quite, old, matter, many...",1.0,pos
3,4807,"[really, thought, movie, superb, history, corr...",1.0,pos
4,5258,"[ladies, man, laugh, loud, funny, great, diver...",1.0,pos


In [ ]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,5739,"[paul, naschy, ghostly, security, guard, scari...",0.7,pos
1,10964,"[many, ways, filmic, career, independent, lege...",0.7,pos
2,4807,"[seen, hong, kong, films, time, say, film, amo...",1.0,pos
3,7259,"[movie, almost, everything, action, cool, funn...",1.0,pos
4,11405,"[need, great, house, babysitter, phone, simon,...",0.8,pos


In [ ]:
print("""TRAIN\n\t - le nombre total de phrases d’entrainement: %d
              \n\t - le nombre total de phrases d’entrainement positives: %d
              \n\t - le nombre total de phrases d’entrainement négatives: %d\n""" % 
      (train_df.shape[0], train_df[train_df["sentiment"] == "pos"].shape[0], 
       train_df[train_df["sentiment"] == "neg"].shape[0]))

print("""TEST\n\t - le nombre total de phrases de test: %d
              \n\t - le nombre total de phrases de test positives: %d
              \n\t - le nombre total de phrases de test négatives: %d\n""" % 
      (test_df.shape[0], test_df[test_df["sentiment"] == "pos"].shape[0], 
       test_df[test_df["sentiment"] == "neg"].shape[0]))

TRAIN
	 - le nombre total de phrases d’entrainement: 25000
              
	 - le nombre total de phrases d’entrainement positives: 12500
              
	 - le nombre total de phrases d’entrainement négatives: 12500

TEST
	 - le nombre total de phrases de test: 25000
              
	 - le nombre total de phrases de test positives: 12500
              
	 - le nombre total de phrases de test négatives: 12500



**b)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence d'au moins 5 et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne) dans le répertoire aclImdb.

In [ ]:
def build_voc(corpus, unk_cutoff=5):
    unigrams = defaultdict(int)
    for review in corpus:
        for word in review:
            unigrams[word] += 1

    print("Unigrammes: " + str(len(unigrams)))

    unigrams = {k: v for k, v in sorted(unigrams.items(), key=lambda item: item[1], reverse=True) if v >= 5}

    print("Taille du vocab (apres filtrage freq > 5): " + str(len(unigrams)))

    f = open('vocab.txt', 'w')
    for w in unigrams.keys():
        f.write(w + '\n')

    return unigrams

In [ ]:
start = time.time()
train_unigrams = build_voc(train_df['text'])
print(time.time() - start)

Unigrammes: 72029
Taille du vocab (apres filtrage freq > 5): 27760
0.6960606575012207


## 2. Matrices de co-occurence (30 points)

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

**a)** A partir des textes du corpus d’entrainement (neg/pos), vous devez construire une matrice de co-occurrence mot × mot M(w,w) qui contient les 5000 unigrammes les plus fréquents sous forme de **cadre panda**. Le contexte de co-occurrence est une fenêtre de +/-5 mots autour du mot cible. Le poids est la fréquence de co-occurrence simple. Sauvegardez votre matrice dans un fichier tp2_mat5.csv dans le répertoire vsm.

Attention, le mot lui même de doit pas être compté dans sa co-occurence. Exemple : 
Corpus: [ "I go to school every day by bus", "i go to theatre every night by bus"]

Co-occurence("every", fenetre=2) = [ (to, 2), (by, 2), (school, 1), (day, 1), (theatre, 1), (night, 1), (bus, 0), (every, 0), (go, 0). (i,0) ]

In [ ]:
train_top_5000 = list(train_unigrams.keys())[:5000] # or read first 5000 lines from vocab.txt
train_5000_df = pd.DataFrame(train_top_5000, columns=['unigram'])
train_5000_df.shape
train_5000_df.head()

,unigram
0,movie
1,film
2,one
3,like
4,good


In [ ]:
def matrix_creator(corpus, vocab, n, scaled) :
    M = {}
    unigrams = []

    for review in corpus :
        for word in review :
            if word not in M :
                M[word] = {}
            unigrams.append(word)

        for i, word in enumerate(review) :
            window = [i+k for k in range(-n,n+1,1) if i+k != i and i+k in range(len(review))]
            voisinage = M[word]
            for j in window :
                if word == review[j]:
                    continue
                if review[j] not in voisinage :
                    voisinage[review[j]] = 0
                voisinage[review[j]] += 1 if not scaled else 1 / abs(i - j)

    return [(k, M[k]) for k in vocab] 

In [ ]:
start = time.time()
result = matrix_creator(train_df['text'], train_top_5000, 5, False)
tmp = pd.DataFrame(result)
Mww = pd.json_normalize(tmp[1])
Mww = Mww[tmp[0]].fillna(0).set_index(tmp[0])
Mww = pd.DataFrame(Mww)
print(time.time() - start)

print(Mww)

150.1029634475708
             movie    film     one    like  ...  region  shoddy  exposition  gender
0                                           ...                                    
movie          0.0  2204.0  3756.0  3763.0  ...     5.0     8.0         6.0     6.0
film        2204.0     0.0  3279.0  2576.0  ...    10.0    16.0        15.0    11.0
one         3756.0  3279.0     0.0  1635.0  ...     6.0     6.0         4.0     3.0
like        3763.0  2576.0  1635.0     0.0  ...     2.0     7.0         4.0     2.0
good        3540.0  2473.0  1331.0   983.0  ...     0.0     4.0         0.0     2.0
...            ...     ...     ...     ...  ...     ...     ...         ...     ...
tyler          4.0     3.0     4.0     5.0  ...     0.0     0.0         0.0     0.0
region         5.0    10.0     6.0     2.0  ...     0.0     0.0         0.0     0.0
shoddy         8.0    16.0     6.0     7.0  ...     0.0     0.0         0.0     0.0
exposition     6.0    15.0     4.0     4.0  ...     0.0   

In [ ]:
pd.DataFrame(Mww).to_csv('Mww.csv')

In [ ]:
start = time.time()
result = matrix_creator(train_df['text'], train_top_5000, 5, True)
tmp = pd.DataFrame(result)
Mww_scaled = pd.json_normalize(tmp[1])
Mww_scaled = Mww_scaled[tmp[0]].fillna(0).set_index(tmp[0])
Mww_scaled = pd.DataFrame(Mww_scaled)
print(time.time() - start)

print(Mww_scaled)

156.1667718887329
                  movie         film  ...  exposition    gender
0                                     ...                      
movie          0.000000   788.200000  ...    2.033333  2.900000
film         788.200000     0.000000  ...    6.100000  4.700000
one         1660.766667  1448.483333  ...    1.333333  1.666667
like        1848.800000  1219.616667  ...    1.450000  0.583333
good        1888.966667  1289.850000  ...    0.000000  0.450000
...                 ...          ...  ...         ...       ...
tyler          1.083333     0.700000  ...    0.000000  0.000000
region         2.250000     4.800000  ...    0.000000  0.000000
shoddy         2.966667     7.933333  ...    0.000000  0.000000
exposition     2.033333     6.100000  ...    0.000000  0.000000
gender         2.900000     4.700000  ...    0.000000  0.000000

[5000 rows x 5000 columns]


In [ ]:
pd.DataFrame(Mww_scaled).to_csv('Mww_scaled.csv')

**b)** Calculez maintenant une matrice de cooccurrence mais en ajustant les fréquences basées sur la proximité du mot cible par exemple en les multipliant par 1/𝑑 où d est la distance en jetons (mots) de la cible. Sauvegardez votre matrice (toujours sous forme de cadre panda) dans un fichier tp2_mat5_scaled.csv dans le répertoire vsm.

Voir fonction dessus

**c)**	Vous devez créer une fonction `pmi` qui prend le cadre panda de la matrice $M(w,w)$ et un paramètre boolean flag qui est à True lorsque l'on désire calculer PPMI et à False quand on veut calculer PMI. La fonction transforme la matrice en entrée en une matrice $M’(w,w)$ avec les valeurs PMI ou PPMI selon la valeur du paramètre booléen. La fonction retourne le nouveau cadre panda correspondant. 

Pour une matrice  $X_{m \times n}$:


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


In [ ]:
def mat_list_to_df(mat_result, unigramme_list):
    arr = np.array(mat_result)
    t_arr = arr.T
    arr_list = t_arr.tolist()
    dict_result = {}
    for i, word in enumerate(unigramme_list):
        dict_result[word] = arr_list[i]
    df = pd.DataFrame(data = dict_result, index = unigramme_list)
    return df

def probability_matrix(mat_Result):
    total = 0
    for line in mat_Result:
        total += sum(line)
    arr = np.array(mat_Result)
    with np.errstate(divide='ignore'):
        arr_prob = np.divide(arr, total)
    return arr_prob.tolist()

def ppmi_pmi(df_Result, is_ppmi):

    temp = df_Result.to_numpy()
    mat_Result = list(temp.tolist())
    mat_prob = probability_matrix(mat_Result)
    arr = np.array(mat_prob)
    sum_arr_column = arr.sum(axis=0)
    sum_arr_line = arr.sum(axis=1)

    for i, line in enumerate(mat_prob):
        for j, item in enumerate(line):
            if item == 0:
                mat_prob[i][j] = 0.0
            elif is_ppmi:         
                mat_prob[i][j] = max(math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2), 0)
            else:
                mat_prob[i][j] = math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2)
    
    return mat_list_to_df(mat_prob, list(df_Result.columns))

**d)** Créer les matrice PMIs et PPMIs en vous basant sur les deux matrices que vous avez déjà créée	Sauvegardez vos matrices dans un fichier tp2_mat5<\_scaled>_{pmi|ppmi}.csv toujours dans le répertoire vsm. 

(votre nom de fichier doit contenir "_scaled" s'il est formé à partir Mww_scaled et "pmi" si le flag est false "ppmi" sinon) 

In [ ]:
start = time.time()
result_ppmi_df = ppmi_pmi(Mww, True)
print(time.time() - start)
print(result_ppmi_df)
result_ppmi_df.to_csv('tp2_mat5_ppmi.csv')

start = time.time()
result_pmi_df = ppmi_pmi(Mww, False)
print(time.time() - start)
print(result_pmi_df)
result_pmi_df.to_csv('tp2_mat5_pmi.csv')

22.27034091949463
               movie      film       one  ...    shoddy  exposition    gender
movie       0.000000  0.000000  0.023057  ...  0.000000    0.000000  0.000000
film        0.000000  0.000000  0.000000  ...  0.652134    0.618209  0.255346
one         0.023057  0.000000  0.000000  ...  0.000000    0.000000  0.000000
like        0.384492  0.000000  0.000000  ...  0.413003    0.000000  0.000000
good        0.698302  0.326841  0.027854  ...  0.007592    0.000000  0.000000
...              ...       ...       ...  ...       ...         ...       ...
tyler       0.000000  0.000000  0.000000  ...  0.000000    0.000000  0.000000
region      0.000000  0.128129  0.000000  ...  0.000000    0.000000  0.000000
shoddy      0.000000  0.652134  0.000000  ...  0.000000    0.000000  0.000000
exposition  0.000000  0.618209  0.000000  ...  0.000000    0.000000  0.000000
gender      0.000000  0.255346  0.000000  ...  0.000000    0.000000  0.000000

[5000 rows x 5000 columns]
20.463147401809692

In [ ]:
start = time.time()
result_ppmi_sc_df = ppmi_pmi(Mww_scaled, True)
print(time.time() - start)
print(result_ppmi_sc_df)
result_ppmi_sc_df.to_csv('tp2_mat5_scaled_ppmi.csv')

start = time.time()
result_pmi_sc_df = ppmi_pmi(Mww_scaled, False)
print(time.time() - start)
print(result_pmi_sc_df)
result_pmi_sc_df.to_csv('tp2_mat5_scaled_pmi.csv')

21.404537677764893
               movie      film      one  ...    shoddy  exposition    gender
movie       0.000000  0.000000  0.00000  ...  0.000000    0.000000  0.000000
film        0.000000  0.000000  0.00000  ...  0.716377    0.427021  0.197189
one         0.000000  0.000000  0.00000  ...  0.000000    0.000000  0.000000
like        0.440225  0.000000  0.00000  ...  0.000000    0.000000  0.000000
good        0.851384  0.455020  0.08143  ...  0.000000    0.000000  0.000000
...              ...       ...      ...  ...       ...         ...       ...
tyler       0.000000  0.000000  0.00000  ...  0.000000    0.000000  0.000000
region      0.000000  0.174144  0.00000  ...  0.000000    0.000000  0.000000
shoddy      0.000000  0.716377  0.00000  ...  0.000000    0.000000  0.000000
exposition  0.000000  0.427021  0.00000  ...  0.000000    0.000000  0.000000
gender      0.000000  0.197189  0.00000  ...  0.000000    0.000000  0.000000

[5000 rows x 5000 columns]
20.893553972244263
          

In [ ]:
result_ppmi_df.head()

,movie,film,one,like,good,would,even,time,really,story,see,much,could,well,get,people,bad,also,great,first,made,way,make,movies,think,characters,character,watch,films,many,two,seen,never,little,acting,plot,best,love,show,life,...,clarke,roommate,sentinel,relax,electric,dysfunctional,tomorrow,choppy,examination,gregory,roof,alec,generated,fingers,ho,complaining,adopted,unlikable,da,bud,matched,alicia,items,muddled,invites,greatness,bernard,filling,knocked,dud,policeman,tacky,les,automatically,ordered,tyler,region,shoddy,exposition,gender
movie,0.000000,0.000000,0.023057,0.384492,0.698302,0.422466,0.241866,0.283324,0.599997,0.000000,0.637430,0.288885,0.361314,0.079987,0.018666,0.193751,0.890248,0.000000,0.609331,0.598753,0.772781,0.045549,0.524690,0.296342,0.635505,0.000000,0.0,1.044950,0.000000,0.115890,0.000000,0.876434,0.203810,0.000000,0.342297,0.398804,0.311948,0.305089,0.000000,0.0,...,0.000000,0.280305,0.457756,1.289101,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.415320,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.616954,0.000000,0.000000,0.000000,0.470113,0.000000,0.000000,0.095717,0.527136,0.091493,0.000000,0.000000,0.000000,0.000000,0.000000
film,0.000000,0.000000,0.000000,0.000000,0.326841,0.225054,0.000000,0.000000,0.230446,0.000000,0.353863,0.325570,0.143639,0.265272,0.000000,0.000000,0.181472,0.054713,0.420084,0.510446,0.623606,0.006928,0.301055,0.000000,0.086710,0.000000,0.0,0.337255,0.089916,0.246923,0.000000,0.519424,0.111339,0.107948,0.029768,0.000000,0.415291,0.000000,0.000000,0.0,...,0.000000,0.000000,0.603781,0.002168,0.0,0.0,0.000000,0.597515,0.092442,0.610977,0.000000,0.141091,0.000000,0.000000,0.000000,0.354895,0.067481,1.010073,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.347943,0.246405,0.448469,0.000000,1.130712,0.000000,0.340448,0.000000,0.580052,0.000000,0.000000,0.128129,0.652134,0.618209,0.255346
one,0.023057,0.000000,0.000000,0.000000,0.027854,0.000000,0.000000,0.355893,0.065921,0.000000,0.071530,0.000000,0.030686,0.000000,0.000000,0.000000,0.177448,0.000000,0.132090,0.365593,0.333980,0.000000,0.038528,1.494353,0.197422,0.011515,0.0,0.220840,1.397679,0.198003,0.637808,1.152363,0.000000,0.000000,0.000000,0.000000,1.421808,0.023471,0.000000,0.0,...,0.000000,0.000000,0.099010,0.000000,0.0,0.0,0.000000,0.000000,0.172633,0.000000,0.403548,0.735856,0.065898,0.000000,0.945716,0.000000,0.000000,0.000000,0.372129,0.327997,0.0,0.0000,0.172633,0.0,0.200598,1.165100,0.000000,0.000000,0.697313,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
like,0.384492,0.000000,0.000000,0.000000,0.000000,0.569553,0.059891,0.000000,0.565380,0.000000,0.240583,0.490832,0.000000,0.000000,0.038478,0.692988,0.278921,0.000000,0.000000,0.000000,0.186521,0.000000,0.346436,1.244465,0.384802,0.087742,0.0,0.215840,0.951823,0.215429,0.000000,0.000000,0.091014,0.094198,0.000000,0.000000,0.000000,0.000000,0.230212,0.0,...,0.533902,0.000000,0.820329,0.000000,0.0,0.0,1.279088,0.358383,0.000000,0.000000,0.000000,0.000000,0.202254,0.538955,0.000000,0.607969,0.000000,0.000000,0.245451,0.271709,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.286506,0.000000,0.084225,0.893952,1.168431,0.610294,0.000000,0.000000,0.071355,0.000000,0.413003,0.000000,0.000000
good,0.698302,0.326841,0.027854,0.000000,0.000000,0.000000,0.097442,0.000000,0.900549,0.587106,0.124358,0.000000,0.031490,0.452498,0.000000,0.000000,1.239403,0.361139,0.406001,0.078291,0.057700,0.108202,0.286359,0.486428,0.476146,0.000000,0.0,0.466959,0.247345,0.000000,0.000000,0.000000,0.000000,0.000000,1.383618,0.241143,0.000000,0.000000,0.282098,0.0,...,0.128491,0.081348,1.374276,0.035697,0.0,0.0,0.417998,0.000000,0.000000,0.000000,0.000000,0.759583,0.019236,0.000000,0.000000,0.331841,0.000000,0.000000,0.000000,0.000000,0.0,0.2169,0.447899,0.0,0.000000,0.000000,0.000000,0.425415,0.000000,0.000000,0.000000,0.000000,0.000000,2.198544,0.000000,0.000000,0.000000

## 3. Test de PPMI (20 points)

Pour le test des matrices de cooccurrences, nous allons comparer deux mesures de distance entre deux vecteurs, la distance euclidienne et la distance cosinus provenant du module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

\\

**a)**	Implémentez la fonction voisins(mot, pd, distance) qui prend un mot en entrée et une métrique de distance et qui retourne les n mots les plus similaires selon la mesure. Pour un mot w, elle ordonne tous les mots du vocabulaire en fonction de leur distance de w en utilisant la métrique de distance distance (par défaut: cosine)sur le vsm pd. Les mesures de distance à tester sont : la distance Euclidienne et la distance cosinus implantées ci-dessus.

In [ ]:
def voisins(word, df, n, distfunc=cosine):
    assert distfunc.__name__ == 'cosine' or distfunc.__name__ == 'euclidean', "distance metric not supported"
    assert word in df.columns, "word not in df"
    order = True if distfunc.__name__ == 'euclidean' else False

    closest = {}
    for w in df.columns:
        if w == word:
            continue
        distance = distfunc(df[word], df[w])
        closest[w] = distance
    closest = {k: v for k, v in sorted(closest.items(), key=lambda item: item[1], reverse=order)}

    return list(closest.keys())[:n], list(closest.values())[:n]

**b)** En utilisant le cadre panda associé aux matrices Mww et Mww scaled, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [ ]:
print(voisins('beautiful', Mww, 5, euclidean))
print(voisins('beautiful', Mww_scaled, 5, cosine))

(['movie', 'film', 'one', 'like', 'good'], [14089.164205161354, 11184.159423041143, 8532.992499703723, 6955.693782794065, 5842.1051000474135])
(['amazing', 'also', 'however', 'wonderful', 'although'], [0.1574254021414464, 0.16056471525151417, 0.16951529676034738, 0.17668250674375796, 0.17743236238854254])


**c)** En utilisant les cadres panda associés aux matrices PMIs, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [ ]:
print(voisins('beautiful', result_pmi_df, 5, euclidean))
print(voisins('beautiful', result_pmi_df, 5, cosine))

(['bad', 'orders', 'officer', 'sends', 'weapon'], [102.04224518453064, 100.32570330509236, 100.17312615612182, 99.92420765152575, 99.72726079811119])
(['wonderful', 'beauty', 'cinematography', 'gorgeous', 'perfect'], [0.6776632941554144, 0.6969494495625018, 0.6994158377456005, 0.7043181934888443, 0.7138009799274339])


**d)** En utilisant les cadres panda associés aux matrices PPMIs, trouvez les 5 mots les plus similaires au mot
« beautiful » et affichez-les, pour chacune des deux distances. 

In [ ]:
print(voisins('beautiful', result_ppmi_df, 5, euclidean))
print(voisins('beautiful', result_ppmi_df, 5, cosine))

(['orders', 'colonel', 'secretary', 'officer', 'assistant'], [88.99850048867745, 88.49787876325419, 88.37442627704499, 88.21623202572202, 88.18776700866722])
(['gorgeous', 'beauty', 'wonderful', 'cinematography', 'woman'], [0.5864706015291566, 0.6018621511627195, 0.6059481605247956, 0.6118684740882059, 0.6272274889134483])


**e)** Que constatez-vous entre la différence de performance de la distance euclidienne et la distance cosinus ? Que constatez-vous entre les différents types de matrices de cooccurrence ?

On remarque qu'avec la distance cosinus, les mots detectés comme les plus similaires sont semantiquement plus proches du mot "beautiful" que ceux retournés par la distance euclidienne. En regardant les matrices de PMI et PPMI, on constate que les scores de PMI donne une meuilleure correspondance entre le mot "beautiful" et ses voisins (4 mots sur 5).

## 4.	Réduction de dimensionnalité (20 points)

**a)** Ecrivez une fonction lsa qui prend en entrée un cadre panda pd (qui contient votre matrice / vsm) et un paramètre K (qui indique le nombre de dimensions finales), et qui applique LSA avec ce paramètre k sur la matrice et retourne le vsm réduit sous forme de cadre panda.

In [ ]:
def lsa(df, k):
    # print(df)
    U, s, V = np.linalg.svd(df, full_matrices=False)
    S = np.diag(s[:k])
    Ck = np.dot(U[:, :k], S)
    df_r =  pd.DataFrame(Ck).T
    df_r.columns = df.keys()
    return  df_r

**b)** Exécutez lsa sur les cadres panda associés à vos matrices Mww et Mww_scaled avec une dimension k=100

In [ ]:
Mww_lsa = lsa(Mww, 100)
Mww_scaled_lsa = lsa(Mww_scaled, 100)

             movie    film     one    like  ...  region  shoddy  exposition  gender
0                                           ...                                    
movie          0.0  2204.0  3756.0  3763.0  ...     5.0     8.0         6.0     6.0
film        2204.0     0.0  3279.0  2576.0  ...    10.0    16.0        15.0    11.0
one         3756.0  3279.0     0.0  1635.0  ...     6.0     6.0         4.0     3.0
like        3763.0  2576.0  1635.0     0.0  ...     2.0     7.0         4.0     2.0
good        3540.0  2473.0  1331.0   983.0  ...     0.0     4.0         0.0     2.0
...            ...     ...     ...     ...  ...     ...     ...         ...     ...
tyler          4.0     3.0     4.0     5.0  ...     0.0     0.0         0.0     0.0
region         5.0    10.0     6.0     2.0  ...     0.0     0.0         0.0     0.0
shoddy         8.0    16.0     6.0     7.0  ...     0.0     0.0         0.0     0.0
exposition     6.0    15.0     4.0     4.0  ...     0.0     0.0         0.0 

In [ ]:
print(Mww_lsa)
print(Mww_scaled_lsa)

**c)** En utilisant les matrices de co-occurrence (de base et scalés) réduites avec LSA, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les.

In [ ]:
print(voisins('beautiful', Mww_lsa, 5, cosine))
print(voisins('beautiful', Mww_scaled_lsa, 5, cosine))

(['beauty', 'gorgeous', 'amazing', 'lovely', 'moving'], [0.032669409202906174, 0.03657954413484443, 0.04208923847784096, 0.042402514897163956, 0.04251957653355276])
(['gorgeous', 'lovely', 'powerful', 'beauty', 'moving'], [0.05065235807436552, 0.05951329170048869, 0.05961045482887939, 0.06405762226993739, 0.07215139001822224])


d) En utilisant les matrices PMIs et PPMIs réduites avec lsa, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les

In [ ]:
ppmi_lsa = lsa(result_ppmi_df, 100)
pmi_lsa = lsa(result_pmi_df, 100)
ppmi_lsa_scaled = lsa(result_ppmi_sc_df, 100)
pmi_lsa_scaled = lsa(result_pmi_sc_df, 100)

print(voisins('beautiful', ppmi_lsa, 5, cosine))
print(voisins('beautiful', ppmi_lsa_scaled, 5, cosine))
print(voisins('beautiful', pmi_lsa, 5, cosine))
print(voisins('beautiful', pmi_lsa_scaled, 5, cosine))

(['gorgeous', 'beauty', 'lovely', 'wonderful', 'stunning'], [0.12760579614031764, 0.13771878225400447, 0.1755551581903143, 0.18865325555909485, 0.18935457217382423])
(['gorgeous', 'lovely', 'stunning', 'beauty', 'wonderful'], [0.1150025841222665, 0.16724701757559501, 0.17983857142273496, 0.19269401657335683, 0.20377703421375237])
(['wonderful', 'nice', 'beauty', 'perfect', 'amazing'], [0.24793610676783162, 0.277462073128817, 0.2899269744357047, 0.29695605685101367, 0.3178627577431469])
(['wonderful', 'nice', 'amazing', 'gorgeous', 'beauty'], [0.2946922586535222, 0.36872790968350555, 0.3912010451738721, 0.40424849751648173, 0.413443756722526])


**e)** En utilisant sklearn.decomposition.TruncatedSVD, créez les matrices réduites à partir des mêmes matrices que celles de la question précédentes (la matrice pmi et la matrice pmi_scaled) Puis tester ces nouvelles matrices LSA pour trouver les 5 mots les plus similaires au mot « beautiful » 

Ici aussi, nous voulons aussi obtenir des matrices de dimension k=100

In [ ]:
def sklearn_svd(df, k):
    svd_model = TruncatedSVD(n_components=k)
    df_r = svd_model.fit_transform(df)
    df_r =  pd.DataFrame(df_r).T
    df_r.columns = df.keys()
    return  df_r

In [ ]:
ppmi_svd = sklearn_svd(result_ppmi_df, 100)
pmi_svd = sklearn_svd(result_pmi_df, 100)
ppmi_svd_scaled = sklearn_svd(result_ppmi_sc_df, 100)
pmi_svd_scaled = sklearn_svd(result_pmi_sc_df, 100)

print(voisins('beautiful', ppmi_svd, 5, cosine))
print(voisins('beautiful', ppmi_svd_scaled, 5, cosine))
print(voisins('beautiful', pmi_svd, 5, cosine))
print(voisins('beautiful', pmi_svd_scaled, 5, cosine))

(['gorgeous', 'beauty', 'stunning', 'wonderful', 'lovely'], [0.13635057215259017, 0.14477190292361775, 0.1739820015877136, 0.17523504033250148, 0.18091811565242333])
(['gorgeous', 'lovely', 'beauty', 'stunning', 'wonderful'], [0.11110285774789697, 0.16167138914130685, 0.18842995122898665, 0.19146061866476305, 0.19259086331273823])
(['wonderful', 'nice', 'perfect', 'beauty', 'amazing'], [0.24238432439533164, 0.27146541720778616, 0.2891781822647449, 0.307952544809302, 0.30884441201256585])
(['wonderful', 'nice', 'amazing', 'beauty', 'perfect'], [0.2863561546786706, 0.34021860017258165, 0.3925005691837269, 0.41003174806305653, 0.4172124514280279])


f) Commentez vos résultats

Les 2 methodes LSA et SVD detectent les memes mots similaires a "beautiful", c'est juste l'ordre des mots qui varient (et les scores/distances associees different legerement).

## 5. Évaluation (10 points)

Il est temps d’évaluer l’intérêt de nos modèles de vecteurs. Nous allons pour cela utiliser un ensemble de données de similarité de mots (relatedness) The MEN Test Collection, qui se trouve dans le répertoire test. L’ensemble de données contient une paire de mots avec un score de similarité attribué par des humains. En d’autres termes, une ligne (un exemple) de l’ensemble de données est de la forme : \<mot_1> \<mot_2> \<score>.

Pour aligner les distances obtenues avec vos métriques, ce score est converti en nombre réel négatif par la fonction read_test_dataset que vous avez dans le squelette du TP.

La métrique d’évaluation est le coefficient de corrélation de Spearman 𝜌 entre les scores humains et vos distances (voir https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). 

Nous allons maintenant évaluer les différents vsm obtenus sur l'ensemble de données: MEN_dataset.


#### Fonctions pour lire le jeu de données MEN

In [ ]:
def read_test_dataset(
        src_filename,
        delimiter=','):
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[2]
            score = -float(score)
            yield (w1, w2, score)

In [ ]:
# Retourne un itérable sur le jeu de données MEN
def men_dataset():
    src_filename = os.path.join(
        DIRNAME_MEN, 'MEN_dataset_natural_form_full')
    return read_test_dataset(
        src_filename, delimiter=' ')

In [ ]:
def evaluate(ds, df, distfunc=cosine):
    """
    ds : iterator
       retourne des tuples (word1, word2, score).

    df : pd.DataFrame
        le modèle vsm à évaluer

    distfunc : la mesure de distance entre vecteurs
  
    Retour: le coefficient de correlation de Spearman entre les scores de l'ensemble de données de test 
    et celui du modele vsm qui se présente sous la forme d'un cadre Panda pd avec les colonnes
    ['word1', 'word2', 'score', 'distance'].
    """
    data = []
    for w1, w2, score in ds():
        d = {'word1': w1, 'word2': w2,'score': score}
        if w1 not in df.index or w2 not in df.index:
            continue
        else:
            w1 = df.loc[w1]
            w2 = df.loc[w2] 
        d['distance'] = distfunc(w1, w2)
        data.append(d)

    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, b=data['distance'].values)
    return rho, data

**a)**	Testez chacun de vos modèles vsm (Matrice de base, matrice scalée, les PMIs et PPMIs et toutes les matrices LSA (de base, scalée, pmi, ppmi) en appelant la fonction evaluate avec les deux mesure de distance (euclidienne et cosinus) et affichez vos résultats dans une seule table.

In [ ]:
vsm_models = [Mww, Mww_scaled, result_pmi_df, result_ppmi_df, result_pmi_sc_df, result_ppmi_sc_df] # pmi_lsa, ppmi_lsa, pmi_lsa_scaled, ppmi_lsa_scaled
for model in vsm_models:
    rho, data = evaluate(men_dataset, model, distfunc=cosine)
    print(rho, data)

for model in vsm_models:
    rho, data = evaluate(men_dataset, model, distfunc=euclidean)
    print(rho, data)

0.004063860230346588         word1        word2  score  distance
0       river        water  -49.0  0.327278
1        rain        storm  -49.0  0.559845
2       dance      dancers  -49.0  0.412789
3      camera  photography  -49.0  0.300166
4       sunny     sunshine  -48.0  0.655110
..        ...          ...    ...       ...
925       car       tongue   -4.0  0.442126
926      fish      theatre   -3.0  0.509976
927       hot       zombie   -3.0  0.253801
928  children         ford   -3.0  0.427045
929     grave          hat   -2.0  0.542818

[930 rows x 4 columns]
0.07551986097318225         word1        word2  score  distance
0       river        water  -49.0  0.482563
1        rain        storm  -49.0  0.736418
2       dance      dancers  -49.0  0.554667
3      camera  photography  -49.0  0.538064
4       sunny     sunshine  -48.0  0.830768
..        ...          ...    ...       ...
925       car       tongue   -4.0  0.639554
926      fish      theatre   -3.0  0.680656
927       h

**b)**	Commentez vos résultats d'évaluation

On remarque qu'on a toujours les memes scores de similarite.